# Terrain Modification Profile Generator
Author:  William Katzenmeyer, P.E., C.F.M.

## Purpose

## Instructions for Input Files





## User Inputs

### User-Defined Variables

| Variable                | Description                                      |
|-------------------------|--------------------------------------------------|
| Profile_Segment_Length  | Length of Segment for Lowest Point Groupings (in meters) |
| Profile_Vertical_Adjust | Vertical Adjustment Applied to Lowest Points = -0.25 |
| Sampling_Interval       | Raster Sampling Interval (in meters)             |
| Projection_File_Path    | HEC-RAS RASMapper Projection File                |

### File Paths for Terrain Modifications and Terrain TIFFs

| Variable          | Description                                  |
|-------------------|----------------------------------------------|
| RASTerrainMods    | GEOJSON containing terrain modification polylines |
| RASTerrainTiff1   | HEC-RAS RASMapper Terrain TIFF file          |

## Script Output: 

The script will produce a CSV which contains a profile for each terrain modification polyline.  These can by copy/pasted into the terrain modification editor in RASMapper

Example:
| PolylineID      | STA      | Elevation |
|-----------------|----------|-----------|
| Profile Line 1  | 0        | 43.46875  |
| Profile Line 1  | 2963.51  | 33.21875  |
| Profile Line 1  | 5994.47  | 30        |
| Profile Line 1  | 8644.55  | 28.875    |
| Profile Line 1  | 11584.35 | 26.46875  |
| Profile Line 1  | 14838.87 | 25.125    |
| Profile Line 2  | 0        | 43.46875  |
| Profile Line 2  | 2963.51  | 33.21875  |
| Profile Line 2  | 5994.47  | 30        |
| Profile Line 2  | 8644.55  | 28.875    |
| Profile Line 2  | 11584.35 | 26.46875  |
| Profile Line 2  | 14838.87 | 25.125    |



In [ ]:
# Define user-defined variables
Profile_Segment_Length = 3000  # (in meters)
Profile_Vertical_Adjust = -0.25
Sampling_Interval = 1/3.2808  # User-defined sampling interval in meters
Projection_File_Path = r"C:\WMK_Working\WF_WestForkCalcasieu\RAS2D_template\NAD_1983_2011_StatePlane_Louisiana_South_FIPS_1702_Ft_US.prj"

# Specify file paths for terrain modifications and terrain TIFFs
RASTerrainMods = r"C:\WMK_Working\TerrainModsPolyline.geojson"
RASTerrainTiff1 = r"C:\WMK_Working\WF_WestForkCalcasieu\RAS2D_template\Terrain\WF_Terrain_Final.WF_Terrain_Base.tif"

In [ ]:
# Install required libraries using Pip
import subprocess
import sys

def install_and_import(package_name, import_as=None):
    try:
        if import_as:
            exec(f"import {package_name} as {import_as}")
            print(f"{package_name} (as {import_as}) is already installed.")
        else:
            __import__(package_name)
            print(f"{package_name} is already installed.")
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installed successfully.")
        if import_as:
            exec(f"import {package_name} as {import_as}")
        else:
            __import__(package_name)

# List of packages and their import names
packages = {
    "geopandas": "gpd",
    "rioxarray": None,
    "pandas": "pd",
    "numpy": "np",
    "shapely": None,
    "ipywidgets": "widgets",
    "pyproj": None
}

# Install and import packages
for package, import_as in packages.items():
    install_and_import(package, import_as)

# Special case for IPython display, as it's part of the IPython core package
try:
    from IPython.display import display
    print("IPython display is already available.")
except ImportError:
    print("Installing IPython...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "IPython"])
    from IPython.display import display
    print("IPython installed successfully.")
import geopandas as gpd
import rioxarray
import pandas as pd
import numpy as np
import shapely.geometry
from IPython.display import display
import ipywidgets as widgets
from pyproj import CRS


In [ ]:
# Generate Terrain Moodification Profiles for Low Detail/LIDAR Channels and Prepare HEC-RAS Plan File for Floodplain Mapping

try:
    # Load the terrain modifications polyline GeoJSON file
    TerrainModsPolyline = gpd.read_file(RASTerrainMods)
except Exception as e:
    raise Exception("Failed to load the terrain modifications polyline GeoJSON file. Please check the RASMapper Layer and ensure no zero length lines exist.  Error: {e}") from e

# Read the PRJ file to get the projection information
with open(Projection_File_Path, 'r') as file:
    prj_text = file.read()
crs_target = CRS.from_wkt(prj_text)

# Reproject the GeoJSON file to match the CRS of the TIFF files
TerrainModsPolyline = TerrainModsPolyline.to_crs(crs_target.to_string())

# Define the function to read terrain data from a TIFF file
def read_terrain_data_from_tiff(tiff_path, geometry):
    # Open the terrain TIFF file using rioxarray
    terrain_data = rioxarray.open_rasterio(tiff_path)

    # Function to sample terrain data along a line at regular intervals
    def sample_terrain(line, terrain_data):
        points = []
        distance = 0
        previous_coord = None
        for coord in line.coords:
            if previous_coord is not None:
                # Calculate 2D distance between the current and previous coordinate
                distance += np.sqrt((coord[0] - previous_coord[0])**2 + (coord[1] - previous_coord[1])**2)
            point = shapely.geometry.Point(coord)
            elevation = terrain_data.sel(x=point.x, y=point.y, method="nearest").values
            if not np.isnan(elevation):
                points.append({'STA': distance, 'Elevation': elevation[0]})
            previous_coord = coord
        return points

    # Sample the raster at points along the geometry at specified intervals
    terrain_profile_points = []
    if isinstance(geometry, shapely.geometry.LineString):
        terrain_profile_points = sample_terrain(geometry, terrain_data)
    elif isinstance(geometry, shapely.geometry.MultiLineString):
        for line in geometry.geoms:
            terrain_profile_points.extend(sample_terrain(line, terrain_data))
    else:
        raise TypeError("Unsupported geometry type. Expected LineString or MultiLineString.")

    return terrain_profile_points


def post_process_terrain_profiles(terrain_profile_points):
    """
    Post-process the terrain profiles according to specified rules.
    """
    print("Starting post-processing of terrain profiles.")
    # print(f"Input terrain profile points: {terrain_profile_points}")
    
    # Adjust the first and last point
    if terrain_profile_points:
        terrain_profile_points[0]['Elevation'] += Profile_Vertical_Adjust
        terrain_profile_points[-1]['Elevation'] += Profile_Vertical_Adjust
        print(f"Adjusted first point elevation to {terrain_profile_points[0]['Elevation']}")
        print(f"Adjusted last point elevation to {terrain_profile_points[-1]['Elevation']}")

    # Segment the profile and find the lowest elevation in each segment
    processed_points = []
    previous_lowest_elevation = None
    segment_length_in_feet = Profile_Segment_Length * 3.28084  # Convert segment length to feet
    current_sta = 0
    print(f"Segment length in feet: {segment_length_in_feet}")
    
    # Process each segment of the terrain profile
    # While the current station is less than the maximum station in the terrain profile points
    while current_sta < max(point['STA'] for point in terrain_profile_points) * 3.28084:
        # Segment the terrain profile points based on the current station and segment length
        segment_points = [point for point in terrain_profile_points if current_sta <= point['STA'] * 3.28084 < current_sta + segment_length_in_feet]
        print(f"Processing segment starting at STA {current_sta / 3.28084} meters ")
        #print(f"with points: {segment_points}")
        
        if segment_points:
            # Calculate the lowest point in the current segment using a lambda function
            lowest_point = min(segment_points, key=lambda x: x['Elevation'])

            print(f"Lowest point in current segment before adjustment: {lowest_point}")
            
            # Adjust the elevation of the lowest point in the current segment
            lowest_point_elevation_adjusted = lowest_point['Elevation'] + Profile_Vertical_Adjust
            # If the lowest point in a segment is higher than the lowest point of the previous segment,
            # use the same value as the previous segment
            if previous_lowest_elevation is not None and lowest_point_elevation_adjusted > previous_lowest_elevation:
                lowest_point_elevation_adjusted = previous_lowest_elevation
            lowest_point['Elevation'] = lowest_point_elevation_adjusted
            # lowest_point['STA'] = current_sta / 3.28084  # Convert STA back to meters for consistency
            processed_points.append(lowest_point)
            print(f"Lowest point in current segment after adjustment: {lowest_point}")
            
            # Update the previous lowest elevation for the next iteration
            previous_lowest_elevation = lowest_point_elevation_adjusted
        current_sta += segment_length_in_feet

    
    # Process first and last 100ft points
    first_100ft_points = [point for point in terrain_profile_points if point['STA'] <= 30.48]  # 100 feet in meters
    last_100ft_start_sta = total_length - 30.48  # Start STA for the last 100ft
    last_100ft_points = [point for point in terrain_profile_points if point['STA'] >= last_100ft_start_sta]

    if first_100ft_points:
        lowest_first_100ft_point = min(first_100ft_points, key=lambda x: x['Elevation'])
        print(f"Lowest point in first 100ft: {lowest_first_100ft_point}")
        lowest_first_100ft_point['STA'] = 0  # Set STA to 0 for the first point
        processed_points.insert(0, lowest_first_100ft_point)  # Insert at the beginning

    if last_100ft_points:
        lowest_last_100ft_point = min(last_100ft_points, key=lambda x: x['Elevation'])
        print(f"Lowest point in last 100ft: {lowest_last_100ft_point}")
        lowest_last_100ft_point['STA'] = total_length  # Set STA to polyline length for the last point
        processed_points.append(lowest_last_100ft_point)  # Append to the end

    print(f"Processed terrain profile points: {processed_points}")
    return processed_points



# Initialize an empty DataFrame for output
Terrain_Profiles_Output = pd.DataFrame(columns=['PolylineID', 'STA', 'Elevation'])

# Process each polyline in the GeoJSON file
for index, row in TerrainModsPolyline.iterrows():
    polyline = row['geometry']
    polyline_id = row['Name']
    total_length = polyline.length  # Get the total length of the polyline

    # Read terrain profile data for the current polyline
    terrain_profile_points = read_terrain_data_from_tiff(RASTerrainTiff1, polyline)
    #print(f"Polyline ID: {polyline_id} | Initial Terrain Profile Points: {len(terrain_profile_points)}")  # Improved debug output

    # Post-process the terrain profile data
    processed_points = post_process_terrain_profiles(terrain_profile_points)
    #print(f"Polyline ID: {polyline_id} | Post-Processed Points: {len(processed_points)}")  # Improved debug output

    # Append processed data to the output DataFrame
    temp_df = pd.DataFrame(processed_points)
    display(temp_df)
    temp_df['PolylineID'] = polyline_id
    Terrain_Profiles_Output = pd.concat([Terrain_Profiles_Output, temp_df], ignore_index=True)

# Convert Station Column from meters to feet (round to nearest 0.01ft) and handle potential TypeError
Terrain_Profiles_Output['STA'] = Terrain_Profiles_Output['STA'].apply(lambda x: round(x, 2) if isinstance(x, float) else x)

# Use ipywidgets and display() for showing the DataFrame
display_widget = widgets.Output()
with display_widget:
    display(Terrain_Profiles_Output)

# Display the widget
display(display_widget)

# Export the processed terrain profiles to a CSV file
output_csv_path = "Terrain_Profiles_Output.csv"
Terrain_Profiles_Output.to_csv(output_csv_path, index=False)

# Print completion message
print(f"Terrain profile processing complete. Output saved to {output_csv_path}.")
